Dolaczenie modulow wykorzystywanych na potrzeby laboratorium:

In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, load_img
import numpy as np
import matplotlib.pyplot as plt
import glob, os

Ponizej zawarto definicje funkcji, ktore mozna wykorzystac do realizacji zadan laboratoryjnych oraz ich przyklady wywolania:

In [ ]:
def read_img(filename, size):
    img = load_img(filename, target_size=size)
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    return img_array

# Example: img = read_img("img.jpg", (299, 299)) - wczytanie pliku img.jpg znajdujacego sie w tym samym folderze co notebook, 
# wynikowy rozmiar obrazu to 299x299 (i trzy kanaly - kodowanie rgb)

def create_dataset(main_folder_path, size):
    X = []
    y = []

    os.chdir(".")
    for file in glob.glob(main_folder_path + "/*/*.jp*"):
        X.append(read_img(file, size))
        y.append(file.split('\\')[1])
    X = np.asarray(X)
    y = np.asarray(y).astype('float32')
    return X, y

# Example: X, y = create_dataset("faces", (299, 299)) - wczytanie zbioru z folderu faces, w folderze znajduja sie dwa pofoldery - o nazwach 0 i 1, 
# a w nich zdjecia w formacie jpeg lub jpg, wynikowy rozmiar obrazów to 299x299 (i trzy kanaly - kodowanie rgb)

def plot_images(X, y, rows=2, cols=5, clustering=False, nntesting=False, y_pred=None):
    # rows - number of classes - 2
    # cols - number of instances in one class - 5 images per class
    fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(14, 7))
    counter = 0
    for row in range(rows):
        for col in range(cols):
            ax[row][col].imshow(X[counter] * 0.5 + 0.5)
            if not clustering and not nntesting:
                ax[row][col].set_title(f"label:{y[counter]}")
            else:
                if clustering:
                    ax[row][col].set_title(f"cluster number:{y[counter]}")
                else:
                    ax[row][col].set_title(f"true:{y[counter]} predicted:{y_pred[counter]}")
            counter += 1
    plt.show()

# Example: 
# plot_images(X, y) - wyswietlenie zbioru treningowego/do grupowania z 10 elementami (5 na klase)
# plot_images(X_test, y_test, rows=2, cols=2) - wyswietlenie zbioru testowego z 4 elementami (2 na klase)
# plot_images(X, kmeans_prediction, clustering=True) - wywolanie do wyswietlenia wynikow grupowania
# plot_images(X_test, y_test, rows=2, cols=2, clustering=False, nntesting=True, y_pred=nn_pred.reshape(nn_pred.shape[0])) - wyswietlenie wynikow klasyfikacji dla zbioru testowego
# plot_images(X, y, clustering=False, nntesting=True, y_pred=nn_pred.reshape(nn_pred.shape[0])) - wyswietlenie wynikow klasyfikacji dla zbioru treningowego

Do zmiennych X, y oraz X_test, y_test nalezy wczytac obrazy ze stworzonych folderow. W X i X_test przechowywane są tabele reprezentujace obrazy, a w y i y_test - labele.

In [ ]:
X, y = create_dataset("faces", (299, 299))
X_test, y_test = create_dataset("test", (299, 299))

Po wczytaniu tablice powinny miec nastepujace wymiary:

In [ ]:
print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

Nastepnie nalezy wyswietlic obrazy w zbiorach:

In [ ]:
print("Train:")
plot_images(X, y)
print("Test:")
plot_images(X_test, y_test, rows=2, cols=2)

## **Grupowanie**

Wczytanie modelu (bedzie traktowany dalej jako model bazowy) i wyswietlenie jego podsumowania. Prosze skorzystac z modeli Xception lub InceptionV3.

In [ ]:
from keras import Model
# Wczytanie wybranego modelu - przyklad - model Xception
model = Xception()

model.summary()

Funkcja summary nie wyswietla jednak nazw warstw, a jedynie typy. W celu wyswietlenia nazw, za pomoca ktorych mozna dostac sie do konkretnych warstw nalezy zapisac je do listy i wyswietlic:

In [ ]:
# zapisanie do listy nazw warstw
model_layers = [layer.name for layer in model.layers]
print(model_layers)

Wykorzystaj wyzej wczytany model podstawowy (np. Xception) i zbuduj model do ekstrakcji cech. Wykorzystaj parametr modelu input oraz funkcje get_layer (nazwe warstwy wybierz z listy warstw powyzej - ma to byc warstwa przed klasyfikatorem - splaszczajaca dane, np. average pooling) wraz z parametrem output. Wykorzystaj te dwie czesci i zbuduj model z pomoca funkcji Model. Model nazwac model_embeddings.

Hint:

https://keras.io/getting_started/faq/#how-can-i-obtain-the-output-of-an-intermediate-layer-feature-extraction

In [ ]:
#TODO
#TODO
#TODO

Wygenerowanie cech (ekstrakcja cech) za pomoca stworzonego modelu:

In [ ]:
pred_intermediate = model_embeddings.predict(X)

Dokonaj predykcji (funkcja predict modelu) i wykorzystaj funkcje decode_predictions z keras.applications.imagenet_utils do zdekodowania 5 najbardziej prawdobodobych predykcji zwroconych przez model i wypisz je (dla wszystkich obrazkow w zbiorze).

In [ ]:
#TODO
#TODO
#TODO

Zmiana ksztaltu tablicy z labelami (aby mozliwy byl zapis do DataFrame):

In [ ]:
y_reshaped = y.reshape(y.shape[0], 1)
print(f"Shape of y_reshaped: {y_reshaped.shape}")

Z wykorzystaniem struktury DataFrame z bibioteki pandas cechy z sieci konwolucyjnej (embeddings/bottlneck features) sa zapisywane wraz z labelem do tablicy - df:

In [ ]:
import pandas as pd
features = [f"feature_{i}" for i in range(0, pred_intermediate.shape[-1])]
features.append("label")
df_values = np.concatenate((pred_intermediate, y_reshaped), axis=-1)
df = pd.DataFrame(df_values, columns=features)

Wyswietlenie zawartosci df:

In [ ]:
df

Zapis do pliku csv:

In [ ]:
df.to_csv("dataset_faces.csv", index=False)

Wykorzystac algorytm kmeans (KMeans z modulu cluster biblioteki sklearn), dopasowac go do danych przechowywanych w pred_intermediate, dokonac predykcji i jej wyniki zapisac do zmiennej kmeans_prediction. 

Hint:

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [ ]:
# TODO
# TODO
# TODO

Wyswietlic wyniki grupowania wraz z obrazami:

In [ ]:
plot_images(X, kmeans_prediction, clustering=True)

Wyswietl workflow i rezultaty uzyskane w programie Orange:

In [ ]:
print("Orange workflow")
img = load_img("orangeworkflow.png")
img_array = img_to_array(img)/255.
plt.figure(figsize = (15,15))
plt.imshow(img_array)
plt.show()

print("t-SNE results:")
img = load_img("tsne.png")
img_array = img_to_array(img)/255.
plt.figure(figsize = (15,15))
plt.imshow(img_array)
plt.show()

print("Raw kmenas results:")
img = load_img("kmeans.png")
img_array = img_to_array(img)/255.
plt.figure(figsize = (15,15))
plt.imshow(img_array)
plt.show()

print("PCA + kmeans results:")
img = load_img("kmeanspca.png")
img_array = img_to_array(img)/255.
plt.figure(figsize = (15,15))
plt.imshow(img_array)
plt.show()

**TODO: Napisac krotkie wnioski:**

## **Klasyfikacja**

Wczytac model bazowy (Xception lub InceptionV3) z parametrem include_top ustawionym na False (klasyfikator dla zbioru ImageNet jest "odcinany"). Ustawic odpowiedni rozmiar warstwy wejsciowej - (299, 299, 3), ustawic parametr weights na imagenet. Model nazwij base_model.

Hint (dla Xception):

https://keras.io/api/applications/xception/

inne modele (nie wszystkie modele sa bezposrednio dostepne w tensorflow.keras.applications):

https://keras.io/api/applications/

**Na potrzeby laboratorium prosze skorzytac z modeli Xception lun InceptionV3.**, oczywiscie mozna takze "pobawic" sie innymi, ale nie jest gwarantowana ich skutecznosc :)

In [ ]:
# TODO
# TODO
# TODO

Sprawic aby parametry modelu bazowego **nie byly trenowane** - sprawdz parametr trainable modelu

Hint:

https://keras.io/guides/transfer_learning/

In [ ]:
# TODO

Z wykorzystaniem modelu bazowego (np. Xception) stworzyc swoj model do klasyfikacji binarnej zdjec. Do stworzenia modelu wykorzystac nastepujace warstwy: Input, GlobalAveragePooling2D i Dense (w warstwie powinien sie znajdowac 1 neuron. Nalezy tu wykorzystac funkcje aktywacji sigmoid - wtedy na wyjsciu zostanie zwrocona liczba z zakresu 0, 1). Model koncowy nalezy stworzyc za pomoca funkcji Model, ktora jako argumenty przyjmuje warstwe wejsciowa i wyjsciowa. Ponizej przedstawiono wyglad modelu koncowego. Koncowy model nazwac new_model - aby mozliwe bylo korzystanie z dostarczonych pozniej komorek - swoj model takze wyswietlic.

Hint:

https://keras.io/guides/transfer_learning/

In [ ]:
# TODO
# TODO
# TODO


# new_model.summary()

Zbudowany model skompilować (funkcja compile, optymalizator: adam, funkcja straty: binary_crossentropy, metryka do sledzenia: accuracy) oraz przetrenowac (funkcja train, podajemy X i y jako argumenty, trening ma trwac 7 epok, ale mozna sprobowac takze inne liczby - nalezy obserwowac jak zmienia sie accuracy, w zbiorze znajduje sie tylko 10 probek, wiec liczba epok nie powinna byc zbyt duza - wtedy wystepuje szansa na przetrenowanie) - ponizej wynik

In [ ]:
# TODO
# TODO
# TODO

Predykcja za pomoca wytrenowanego modelu:

In [ ]:
nn_pred_train = new_model.predict(X)
print("train raw:")
print(nn_pred_train)
nn_pred_train_binary = np.round(nn_pred_train)
print("train binary:")
print(nn_pred_train_binary)

nn_pred_test = new_model.predict(X_test)
print("test raw:")
print(nn_pred_test)
nn_pred_test_binary = np.round(nn_pred_test)
print("test binary:")
print(nn_pred_test_binary)

Wyznaczenie dokladnosci (accuracy) dla zbioru treningowego i testowego:

In [ ]:
from sklearn.metrics import accuracy_score

train_accuracy = accuracy_score(y, nn_pred_train_binary)
test_accuracy = accuracy_score(y_test, nn_pred_test_binary)

Graficzna prezentacja wynikow i wyswietlenie dokladnosci:

In [ ]:
print("train")
print(f"accuracy: {train_accuracy}")
plot_images(X, y, rows=2, cols=5, clustering=False, nntesting=True, y_pred=nn_pred_train_binary.reshape(nn_pred_train_binary.shape[0]))
print("test")
print(f"accuracy: {test_accuracy}")
plot_images(X_test, y_test, rows=2, cols=2, clustering=False, nntesting=True, y_pred=nn_pred_test_binary.reshape(nn_pred_test_binary.shape[0]))